In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
################################## Import Modules ##################################
# Get Data From Twitter
import GetOldTweets3 as got
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.options import Options
from transformers import ElectraModel, ElectraTokenizer



from pandas import Series, date_range
import time
import re

##### Analysis Data
from transformers import pipeline


In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# fm.get_fontconfig_fonts()
# font_location = '/Library/Fonts/NanumBarunGothicUltraLight 2.ttf' # For Windows
# font_name = fm.FontProperties(fname=font_location).get_name()
# plt.rc('font', family=font_name)
%matplotlib inline


In [7]:
driver_path = "C:/Users/ym/chromedriver/chromedriver.exe"

In [8]:
def run_driver(driver_path):
    options = ChromeOptions()
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("enable-automation")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36")
    driver = Chrome(driver_path, options=options)
    
    return driver

In [9]:
def cleanText(readData):
    text = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', readData)
    return text

In [10]:
def setting_data_format():
    target_word = input('검색할 단어를 입력하세요 : ')
    start_date = input('검색을 시작할 날짜를 입력하세요 : ')
    end_date = input('검색을 종료할 날짜를 입력하세요 : ')
    search_date = date_range(start_date,end_date)

    print(f'''
    데이터 수집 대상 단어 : {target_word}\n
    데이터 수집 시작 일자 : {start_date}\n
    데이터 수집 마감 일자 : {end_date}\n
    데이터 수집 총합 기간 : {len(search_date)}일
    ''')
    return target_word, start_date, end_date


In [70]:
date = datetime.datetime

In [83]:
def crawl_data(driver, target_word):
    new_texts = []
    texts = driver.find_elements_by_xpath("//div[@lang='ko']")
    for text in texts:
        text = text.text
        text = text.replace('\n',' ')
        if target_word not in text:
            continue;
        elif len(text.split(' ')) <= 3:
                continue;
        else:
            text =' '.join( [x  for x in text.split('.') if target_word in x])
            new_texts.append(text)
    return new_texts

In [84]:
def crawl_date(driver, target_date):
    new_date = []
    dates = driver.find_elements_by_xpath("//*[@id='id__o34ppnbhnqe']/time")
    for date in dates:
        date = date.date
        new_date.append(date)
    return new_date

In [86]:
def scroll_down_and_crawl(total_texts, driver, last_height, target_word):
    SCROLL_PAUSE_SEC = 3
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_SEC)
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if last_height != new_height:
        total_texts.extend(crawl_data(driver, target_word))
#         print('지금까지 수집된 데이터 크기 : ',len(total_texts))
        return new_height
    else:
        return False

In [81]:
import datetime # datetime 패키지  
from tqdm.notebook import tqdm 

def crawl_main():
    target_word, start_date, end_date = setting_data_format()
    total_texts = []
    start_time = time.time()
    total_date = []
    
#     days_range = [] # datetime 패키지, datetime 클래스(날짜와 시간 함께 저장)  
#     start = datetime.datetime.strptime("2020-01-01", "%Y-%m-%d") # strptime 메서드: 문자열 반환 
#     end = datetime.datetime.strptime("2020-01-02", "%Y-%m-%d") 
#     date_generated = [start+datetime.timedelta(days=x) for x in range(0, (end-start).days)] 
#     for date in date_generated: days_range.append(date.strftime("%Y-%m-%d")) 
#     print("===설정된 트윗 수집 기간: {} ~ {}===".format(days_range[0], days_range[-1])) 
#     print("===총 {}일 간의 데이터 수집===".format(len(days_range)))

    # 원하는 정보만 저장 # 유저 아이디, 트윗 내용, 날짜, 좋아요 수, 리트윗 수, 링크 
   


    print('=== 데이터 수집 시작 ===')
    driver = run_driver(driver_path)
    url = f'https://twitter.com/search?q="{target_word}"%20since%3A{start_date}%20until%3A{end_date}&src=typed_query&f=live'
    driver.get(url)
    time.sleep(1)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while last_height != False:
        last_height = scroll_down_and_crawl(total_texts, driver, last_height, target_word)
    driver.close()
    print(f'=== 데이터 수집 종료 : {len(total_texts)}===')
    total_texts = list(set(total_texts)) 
    total_texts = [cleanText(text) for text in total_texts]
    total_date = date
    print(f'=== 데이터 정제 완료 : {len(total_texts)}===')
    print(f'=== 데이터 최종 크기 : {len(total_texts)}===')
    print(f'=== 데이터 수집 시간 : {int((time.time() - start_time)//60)}분 ===')
    print('='*30)
    return target_word, start_date, end_date, total_texts, total_date

In [82]:
import pandas as pd
import time
import datetime as dt

def re_judge(result):
    if result['score'] <= 0.33 : 
        return "중립"
    elif result['label'] == "positive":
        return "긍정"
    else:
        return "부정"

def analyze_and_plot(target_word, start_date, end_date, total_texts):
    print('=== 데이터 분석 준비 ===')
    classifier = pipeline('sentiment-analysis',model='monologg/koelectra-base-finetuned-nsmc')
    start_time = time.time()
    print('=== 데이터 분석 시작 ===')
    result = [classifier(text)[0] for text in total_texts]
#     result = [re_judge(re_ju) for re_ju in result]
    print('=== 데이터 분석 종료 ===')
    print(f'=== 데이터 분석 시간 : {int((time.time() - start_time)//60)}분 ===')
    total_dict = pd.DataFrame({'title': total_texts, 'score':result})
    total_dict.to_excel('total.xlsx')
    return(total_dict)
    
#     plt.figure(figsize=(8,8))
#     draw_result = Series(result).value_counts()
#     if len(draw_result) == 2 : 
#         explode = [0,0.1]
#     else:
#         explode = [0,0.1,0.2]
        
#     plt.pie(draw_result,
#                     labels = draw_result.index, 
#                     autopct = "%.1f%%", 
#                     colors = reversed(sns.color_palette('RdBu',2)),
#                     explode = explode
#                    )
#     title = f'트위터 기반 {target_word} 긍정 부정 글의 수 ({start_date} ~ {end_date})'
#     plt.title(title)
#     plt.savefig(title)
#     plt.show()
    
# def analyze_and_df(target_word, start_date, end_date, total_texts):

#     classifier = pipeline('sentiment-analysis',model='monologg/koelectra-base-finetuned-nsmc')
#     start_time = time.time()

#     result = [classifier(text)[0] for text in total_texts]
#     result = [re_judge(re_ju) for re_ju in result]

#     return print(total_texts,result)

def main():
    target_word, start_date, end_date, total_texts = crawl_main()
    analyze_and_plot(target_word, start_date, end_date, total_texts)
if __name__ == "__main__":
    main()

검색할 단어를 입력하세요 : 트럼프
검색을 시작할 날짜를 입력하세요 : 2020-01-01
검색을 종료할 날짜를 입력하세요 : 2020-01-02

    데이터 수집 대상 단어 : 트럼프

    데이터 수집 시작 일자 : 2020-01-01

    데이터 수집 마감 일자 : 2020-01-02

    데이터 수집 총합 기간 : 2일
    
=== 데이터 수집 시작 ===


C:\Users\ym\AppData\Local\Temp/ipykernel_14316/3227642054.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome(driver_path, options=options)
C:\Users\ym\AppData\Local\Temp/ipykernel_14316/3204689472.py:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  texts = driver.find_elements_by_xpath("//div[@lang='ko']")
C:\Users\ym\AppData\Local\Temp/ipykernel_14316/3204689472.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  dates = driver.find_elements_by_xpath("//*[@id='id__o34ppnbhnqe']/time")


=== 데이터 수집 종료 : 274===
=== 데이터 정제 완료 : 245===
=== 데이터 최종 크기 : 245===
=== 데이터 수집 시간 : 1분 ===


ValueError: too many values to unpack (expected 4)

In [ ]:
pip install torch torchvision

In [ ]:
driver = webdriver.Firefox()